In [1]:

from datasets import load_from_disk
import json
import os
import re
from collections import defaultdict

import ast

import pandas as pd
import numpy as np

from typing import List, Callable
from radon.metrics import h_visit
from radon.complexity import cc_visit

layer_indices = [1, 10, 23, 'orig']

In [2]:
subset_name = "processed_Meta-Llama-3.1-8B-Instruct-evals__mbpp__details_results"

# dataset = load_from_disk(f"data/{subset_name}")
df = pd.read_csv("checkpoints/latest_results_clone.csv")

In [3]:
df.head()

,task_type,task_name,subtask_name,input_question,input_choice_list,input_final_prompts,input_correct_responses,output_prediction_text,output_parsed_answer,output_choice_completions,...,trajectories_and_context_10,context_expansion_10,final_output_23,extracted_functions_23,trajectories_and_context_23,context_expansion_23,final_output_orig,extracted_functions_orig,trajectories_and_context_orig,context_expansion_orig
0,Generative,mbpp_chat,NaN,Write a function to check if given tuple is di...,NaN,['<|start_header_id|>user<|end_header_id|>\n\n...,NaN,['\ndef check_distinct(tup):\n return len(tup...,\ndef check_distinct(tup):\n return len(tup) ...,NaN,...,<chain_of_thoughts>\n <thinking>\n ...,"Proposed solution context: Given a tuple, chec...","\ndef check_distinct(numbers):\n """"""Check i...",def check_distinct(numbers):\n return len(n...,<chain_of_thoughts>\n <thinking>\n ...,Proposed solution context: Problem: Write a fu...,"\ndef check_distinct(numbers):\n """"""\n C...",def check_distinct(numbers):\n return len(n...,<chain_of_thoughts>\n <thinking>\n <top...,Proposed solution context: Problem: Check if a...
1,Generative,mbpp_chat,NaN,Write a python function to find the first non-...,NaN,['<|start_header_id|>user<|end_header_id|>\n\n...,NaN,['\ndef first_non_repeating_character(s):\n ...,\ndef first_non_repeating_character(s):\n c...,NaN,...,<chain_of_thoughts>\n <thinking>\n ...,Proposed solution context: def first_non_repea...,\ndef first_non_repeating_character(s):\n c...,def first_non_repeating_character(s):\n cha...,<chain_of_thoughts>\n <thinking>\n ...,Proposed solution context: The problem is to f...,"\ndef first_non_repeating_character(s):\n ""...",def first_non_repeating_character(s):\n cha...,<chain_of_thoughts>\n <thinking>\n ...,Proposed solution context: The problem is to f...
2,Generative,mbpp_chat,NaN,Write a function to check whether the given st...,NaN,['<|start_header_id|>user<|end_header_id|>\n\n...,NaN,['\nimport re\ndef check_char(s):\n if re.m...,\nimport re\ndef check_char(s):\n if re.mat...,NaN,...,<chain_of_thoughts>\n <thinking>\n ...,Proposed solution context: Problem: Check if a...,"\ndef check_char(s):\n """"""Returns 'Valid' i...",def check_char(s):\n if len(set(s)) == 1:\n...,<chain_of_thoughts>\n <thinking>\n ...,Proposed solution context: def check_char(s):\...,"\ndef check_char(s):\n """"""\n This functi...",def check_char(s):\n s = s.lower()\n if ...,<chain_of_thoughts>\n <thinking>\n ...,Proposed solution context: The problem stateme...
3,Generative,mbpp_chat,NaN,Write a function to perform index wise additio...,NaN,['<|start_header_id|>user<|end_header_id|>\n\n...,NaN,"['\ndef add_nested_tuples(tup1, tup2):\n retu...","\ndef add_nested_tuples(tup1, tup2):\n return...",NaN,...,<chain_of_thoughts>\n <thinking>\n ...,Proposed solution context: Problem: Write a fu...,"\ndef add_nested_tuples(tup1, tup2):\n if l...","def add_nested_tuples(tup1, tup2):\n if len...",<chain_of_thoughts>\n <thinking>\n ...,Proposed solution context: Problem: Write a fu...,"\ndef add_nested_tuples(tup1, tup2):\n """"""\...","def add_nested_tuples(tup1, tup2):\n return...",<chain_of_thoughts>\n <thinking>\n ...,Proposed solution context: Problem: Write a fu...
4,Generative,mbpp_chat,NaN,Write a function to compute the value of ncr%p.,NaN,['<|start_header_id|>user<|end_header_id|>\n\n...,NaN,"['\ndef ncr_modp(n, r, p):\n if n < r:\n ...","\ndef ncr_modp(n, r, p):\n if n < r:\n ...",NaN,...,<chain_of_thoughts>\n <thinking>\n ...,Proposed solution context: Problem: Calculate ...,"\nimport math\n\ndef ncr_modp(n, r, p):\n i...","import math\n\ndef ncr_modp(n, r, p):\n if ...",<chain_of_thoughts>\n <thinking>\n ...,Proposed solution context: Problem: Calculate ...,"\ndef ncr_modp(n, r, p):\n if r > n - r: \n...","def ncr_modp(n, r, p):\n if r > n - r: \n ...",<chain_of_thoughts>\n <thinking>\n ...,Proposed solution context: Problem: Calculate ...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 36 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   task_type                               250 non-null    object 
 1   task_name                               250 non-null    object 
 2   subtask_name                            0 non-null      float64
 3   input_question                          250 non-null    object 
 4   input_choice_list                       0 non-null      float64
 5   input_final_prompts                     250 non-null    object 
 6   input_correct_responses                 0 non-null      float64
 7   output_prediction_text                  250 non-null    object 
 8   output_parsed_answer                    250 non-null    object 
 9   output_choice_completions               0 non-null      float64
 10  output_choice_negative_log_likelihoods  0 non-null      float6

In [5]:
def code_length(func: str) -> dict:
    lines = func.split("\n")
    return {"lines": len(lines), "characters": len(func)}


def comment_analysis(func: str) -> dict:
    lines = func.split("\n")
    comment_lines = sum(1 for line in lines if line.strip().startswith("#"))
    return {
        "comment_lines": comment_lines,
        "comment_ratio": comment_lines / len(lines) if lines else 0,
    }

def cyclomatic_complexity(func: str) -> int:
    try:
        return max((cc.complexity for cc in cc_visit(func)), default=0)
    except:
        return 0
def syntactic_correctness(func: str) -> bool:
    try:
        ast.parse(func)
        return True
    except SyntaxError:
        return False
        
def extract_function_name(text):
    if pd.isna(text):
        return None
    pattern = r"(?:def|class|assert)\s+([\w\d_]+)\s*\("
    match = re.search(pattern, text)
    return match.group(1) if match else None
def check_function_name_consistency(test_cases: str, generated_function: str) -> float:
    test_func_name = extract_function_name(test_cases)
    generated_func_name = extract_function_name(generated_function)

    if test_func_name and generated_func_name:
        if test_func_name == generated_func_name:
            pass
            # print("yes equal: ", test_func_name, generated_func_name)
        else:
            if generated_function.lower().__contains__(test_func_name):
                return float(generated_function.lower().__contains__(test_func_name))
            # print(f"not equal: tests; {test_func_name}\ngens: {generated_func_name}, \nbut is contained? {float(generated_function.lower().__contains__(test_func_name))}")
        
        return float(test_func_name == generated_func_name)

    if generated_function.lower().__contains__(test_func_name.lower()):
        return float(generated_function.lower().__contains__(test_func_name))
        # print(f"not equal: tests; {test_func_name}\ngens: {generated_func_name}, \nbut is contained? {float(generated_function.lower().__contains__(test_func_name))}")
        
    # print("were not equal: ", test_func_name, generated_func_name)
    return 0.0
    
def calculate_metrics(row: pd.Series, layer_idx: str) -> dict:
    # func = row[f"extracted_functions_{layer_idx}"]
    func = row[f"final_output_{layer_idx}"]
    func = row[f"final_output_{layer_idx}"]
    # if func:
        # print("func: ", func)
        # print("func: ", row["extracted_test_cases"])
        # print("=" * 100)
    if pd.isna(func):
        return {
            f"{metric}_{layer_idx}": np.nan
            for metric in [
                "syntactic_correctness",
                "cyclomatic_complexity",
                "function_name_consistency",
                "lines",
                "characters",
                "comment_lines",
                "comment_ratio",
                "h1",
                "h2",
                "N1",
                "N2",
                "vocabulary",
                "length",
                "volume",
                "difficulty",
                "effort",
            ]
        }
    # print(
    #     f"for layer idx {layer_idx}: ",
    #     check_function_name_consistency(row["extracted_test_cases"], func),
    # )
    metrics = {
        f"syntactic_correctness_{layer_idx}": int(syntactic_correctness(func)),
        f"cyclomatic_complexity_{layer_idx}": cyclomatic_complexity(func),
        f"function_name_consistency_{layer_idx}": check_function_name_consistency(
            row["extracted_test_cases"], func
        ),
    }

    length_metrics = code_length(func)
    metrics.update({f"{k}_{layer_idx}": v for k, v in length_metrics.items()})

    comment_metrics = comment_analysis(func)
    metrics.update({f"{k}_{layer_idx}": v for k, v in comment_metrics.items()})

    try:
        h_metrics = h_visit(func)
        metrics.update({f"{k}_{layer_idx}": v for k, v in h_metrics.items()})
    except:
        metrics.update(
            {
                f"{k}_{layer_idx}": np.nan
                for k in [
                    "h1",
                    "h2",
                    "N1",
                    "N2",
                    "vocabulary",
                    "length",
                    "volume",
                    "difficulty",
                    "effort",
                ]
            }
        )

    return metrics

def process_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    new_df = df.copy()

    for layer_idx in ["1", "10", "23", "orig"]:
        new_df[f"response_rate_{layer_idx}"] = (
            new_df[f"extracted_functions_{layer_idx}"].notna().astype(int)
        )

        metrics = new_df.apply(lambda row: calculate_metrics(row, layer_idx), axis=1)
        # metrics = pd.DataFrame(
        #     [calculate_metrics(row, layer_idx) for _, row in new_df.iterrows()]
        # )
        for metric in metrics.iloc[0].keys():
            new_df[metric] = metrics.apply(lambda x: x[metric])

    return new_df
new_df = process_dataframe(df)
print(str(new_df.columns))


Index(['task_type', 'task_name', 'subtask_name', 'input_question',
       'input_choice_list', 'input_final_prompts', 'input_correct_responses',
       'output_prediction_text', 'output_parsed_answer',
       'output_choice_completions',
       ...
       'comment_ratio_orig', 'h1_orig', 'h2_orig', 'N1_orig', 'N2_orig',
       'vocabulary_orig', 'length_orig', 'volume_orig', 'difficulty_orig',
       'effort_orig'],
      dtype='object', length=104)


In [6]:
# df = pd.read_csv("structured_checkpoint_data.csv")
# dfnew = pd.read_csv("codellm_metrics.csv")

# Save the new DataFrame
new_df.to_csv("codellm_metrics_notebook_v.csv", index=False)
print(new_df.info(max_cols=150))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 104 columns):
 #    Column                                  Non-Null Count  Dtype  
---   ------                                  --------------  -----  
 0    task_type                               250 non-null    object 
 1    task_name                               250 non-null    object 
 2    subtask_name                            0 non-null      float64
 3    input_question                          250 non-null    object 
 4    input_choice_list                       0 non-null      float64
 5    input_final_prompts                     250 non-null    object 
 6    input_correct_responses                 0 non-null      float64
 7    output_prediction_text                  250 non-null    object 
 8    output_parsed_answer                    250 non-null    object 
 9    output_choice_completions               0 non-null      float64
 10   output_choice_negative_log_likelihoods  0 non-nu

In [47]:
def extract_pure_python_code(text):
    # Extract imports
    import_pattern = r"^(?:from\s+[\w.]+\s+import\s+(?:[\w.]+(?:\s*,\s*[\w.]+)*|\*)|import\s+(?:[\w.]+(?:\s*,\s*[\w.]+)*))(?:\s+as\s+[\w.]+)?"
    imports = re.findall(import_pattern, text, re.MULTILINE)

    # Extract functions
    function_pattern = r"(def\s+\w+\s*\(.*?\):(?:\s*['\"][\s\S]*?['\"])?\s*(?:(?!def\s)[\s\S])*?(?=\ndef|\Z))"
    function_pattern = r"(def\s+\w+\s*\([^)]*\):\s*(?:(?:'''[\s\S]*?'''|\"\"\"[\s\S]*?\"\"\")?\s*[\s\S]*?)(?=\ndef|\Z))"
    function_pattern = r"(def\s+\w+\s*\([^)]*\):(?:\s*(?:\"\"\"[\s\S]*?\"\"\"|'''.*?''')?\s*)?[\s\S]*?(?=\ndef\s|\Z))"
    function_pattern = '# MODE:\n(.*?)\}'
    # function_pattern = r"def\s+\w+\s*\([^)]*\):\s*(?:(?:'''[\s\S]*?'''|\"\"\"[\s\S]*?\"\"\")?\s*[\s\S]*?(?=\ndef|\Z))"

    functions = re.findall(function_pattern, text, re.MULTILINE | re.DOTALL)

    def clean_code(code):
        # Remove docstrings
        # code = re.sub(r'"""[\\s\\S]*?"""|\'\'\'[\\\s\\S]*?\'\'\'|\'''[\\s\\S]*?\'''', "", code)
        code = re.sub(r'"""[\\s\\S]*?"""|\'\'\'[\\s\\S]*?\'\'\'', "", code)

        # Remove comments
        code = re.sub(r"#.*", "", code)
        # Remove empty lines and trailing whitespace
        code = "\n".join(line for line in code.splitlines() if line.strip())
        return code

    cleaned_imports = [clean_code(imp) for imp in imports]
    cleaned_functions = [clean_code(func) for func in functions]

    # Combine imports and functions
    cleaned_code = "\n".join(cleaned_imports)
    if cleaned_imports and cleaned_functions:
        cleaned_code += "\n\n"
    cleaned_code += "\n\n".join(cleaned_functions)

    return cleaned_code

def extract_pure_python_code(text):

    lines = text.split('\n')
    functions = []
    current_function = []
    indent_level = 0
    in_function = False
    for line in lines:
        stripped_line = line.strip()
        
        # Check for function or class definition
        if re.match(r'^(def|class)\s', stripped_line):
            if in_function:
                # Save the previous function if we're starting a new one
                functions.append('\n'.join(current_function))
                current_function = []
            in_function = True
            indent_level = len(line) - len(line.lstrip())
            current_function.append(line)
        elif in_function:
            # Check if this line is part of the current function
            if line.strip() == '' or len(line) - len(line.lstrip()) > indent_level:
                current_function.append(line)
            else:
                # This line is not part of the function, so the function has ended
                functions.append('\n'.join(current_function))
                current_function = []
                in_function = False
                if re.match(r'^(def|class)\s', stripped_line):
                    # If this is a new function definition, start a new function
                    in_function = True
                    indent_level = len(line) - len(line.lstrip())
                    current_function.append(line)

    # Add the last function if there is one
    if current_function:
        functions.append('\n'.join(current_function))

    return functions

def extract_python_code_with_comments(text):
    if pd.isna(text):
        return None
    
    # Regex pattern to match Python code blocks
    pattern = r'```(?:python)?\s*\n((?:(?!```).|\n)+)\s*```'
    matches = re.findall(pattern, text, re.MULTILINE | re.DOTALL)
    
    if not matches:
        print("not matches with comments")
        # Fallback pattern for code not wrapped in backticks
        pattern = r'(?:^|\n)((?:import\s+\w+|from\s+\w+\s+import\s+[\w,\s]+|def\s+\w+\s*\(|class\s+\w+[^\n]*:)(?:\n(?!```|import\s+\w+|from\s+\w+\s+import|def\s+\w+\s*\(|class\s+\w+).*)*)'
        matches = re.findall(pattern, text, re.MULTILINE)
    
    if not matches:
        return None
    
    return '\n\n'.join(matches)

def process_dataframe(df):
    new_df = df.copy()
    
    layer_indices = ['1', '10', '23', 'orig']
    
    for layer_idx in layer_indices:
        column_name = f'final_output_{layer_idx}'
        if column_name in new_df.columns:
            # print("pure extraction ..")
            # new_df[f'pure_code_{layer_idx}'] = new_df[column_name].apply(extract_pure_python_code)
            print("with comments extraction ..")
            new_df[f'code_with_comments_{layer_idx}'] = new_df[column_name].apply(extract_pure_python_code)
    
    return new_df
# 
processed_df = process_dataframe(new_df)
print(processed_df[['pure_code_1', 'code_with_comments_1']].head())

def test_code_extraction(df, num_rows=3):
    layer_indices = ['1', '10', '23', 'orig']
    
    for i in range(min(num_rows, len(df))):
        print(f"\nRow {i + 1}:")
        for layer_idx in layer_indices:
            column_name = f'final_output_{layer_idx}'
            if column_name in df.columns:
                text = df.loc[i, column_name]
                if pd.notna(text):
                    extracted_code = extract_pure_python_code(text)
                    print(f"\nLayer {layer_idx}:")
                    print(len(extracted_code) if extracted_code else f"No code extracted: {text}")
                    if extracted_code:
                        for code in extracted_code:
                            print(code)
                else:
                    print(f"\nLayer {layer_idx}: NaN value")
            else:
                print(f"\nLayer {layer_idx}: Column not found")
        print("\n" + "=" * 50)

# Assuming 'new_df' is your DataFrame
# Test the function on the first 3 rows
# test_code_extraction(new_df)
processed_df.info()



with comments extraction ..


<>:10: SyntaxWarning: invalid escape sequence '\}'
<>:10: SyntaxWarning: invalid escape sequence '\}'
/tmp/ipykernel_1040796/2222861860.py:10: SyntaxWarning: invalid escape sequence '\}'
  function_pattern = '# MODE:\n(.*?)\}'
/tmp/ipykernel_1040796/2222861860.py:10: SyntaxWarning: invalid escape sequence '\}'
  function_pattern = '# MODE:\n(.*?)\}'


AttributeError: 'float' object has no attribute 'split'

In [52]:
# Load the DataFrame
metrics_df = pd.read_csv("codellm_metrics_notebook_v.csv")
print(metrics_df.info())
gen_list_len = 169

def generate_latex_table(data, caption, label):
    latex = f"\\begin{{table}}[h]\n\\centering\n\\caption{{{caption}}}\n\\label{{{label}}}\n"
    latex += "\\begin{tabular}{l" + "c" * len(data.columns) + "}\n\\hline\n"
    latex += "Metric & " + " & ".join(data.columns) + " \\\\\n\\hline\n"
    for index, row in data.iterrows():
        latex += (
            f"{index} & "
            + " & ".join([f"{x:.4f}" if isinstance(x, float) else str(x) for x in row])
            + " \\\\\n"
        )
    latex += "\\hline\n\\end{tabular}\n\\end{table}\n"
    return latex


# Response Rates
response_rates = metrics_df[[f"response_rate_{i}" for i in ["1", "10", "23", "orig"]]].mean()
response_rates_data = pd.DataFrame(
    {
        "Layer 1": [response_rates["response_rate_1"], gen_list_len],
        "Layer 10": [response_rates["response_rate_10"], gen_list_len],
        "Layer 23": [response_rates["response_rate_23"], gen_list_len],
        "Original": [response_rates["response_rate_orig"], gen_list_len],
    },
    index=["Response Rate", "Sample Size"],
)
response_rates_latex = generate_latex_table(
    response_rates_data, "Response Rates Across Different Layers", "tab:response_rates"
)

# Code Quality Metrics
code_quality_data = pd.DataFrame(
    {
        "Layer 1": [
            metrics_df["syntactic_correctness_1"].mean(),
            metrics_df["function_name_consistency_1"].mean(),
        ],
        "Layer 10": [
            metrics_df["syntactic_correctness_10"].mean(),
            metrics_df["function_name_consistency_10"].mean(),
        ],
        "Layer 23": [
            metrics_df["syntactic_correctness_23"].mean(),
            metrics_df["function_name_consistency_23"].mean(),
        ],
        "Original": [
            metrics_df["syntactic_correctness_orig"].mean(),
            metrics_df["function_name_consistency_orig"].mean(),
        ],
    },
    index=["Syntactic Correctness", "Function Name Consistency"],
)
code_quality_latex = generate_latex_table(
    code_quality_data,
    "Code Quality Metrics Across Different Layers",
    "tab:code_quality",
)

# Code Complexity Metrics
code_complexity_data = pd.DataFrame(
    {
        "Layer 1": [metrics_df["cyclomatic_complexity_1"].mean()],
        "Layer 10": [metrics_df["cyclomatic_complexity_10"].mean()],
        "Layer 23": [metrics_df["cyclomatic_complexity_23"].mean()],
        "Original": [metrics_df["cyclomatic_complexity_orig"].mean()],
    },
    index=["Cyclomatic Complexity"],
)
code_complexity_latex = generate_latex_table(
    code_complexity_data,
    "Code Complexity Metrics Across Different Layers",
    "tab:code_complexity",
)

# Code Structure Metrics
code_structure_data = pd.DataFrame(
    {
        "Layer 1": [
            metrics_df["lines_1"].mean(),
            metrics_df["characters_1"].mean(),
            metrics_df["comment_lines_1"].mean(),
            metrics_df["comment_ratio_1"].mean(),
        ],
        "Layer 10": [
            metrics_df["lines_10"].mean(),
            metrics_df["characters_10"].mean(),
            metrics_df["comment_lines_10"].mean(),
            metrics_df["comment_ratio_10"].mean(),
        ],
        "Layer 23": [
            metrics_df["lines_23"].mean(),
            metrics_df["characters_23"].mean(),
            metrics_df["comment_lines_23"].mean(),
            metrics_df["comment_ratio_23"].mean(),
        ],
        "Original": [
            metrics_df["lines_orig"].mean(),
            metrics_df["characters_orig"].mean(),
            metrics_df["comment_lines_orig"].mean(),
            metrics_df["comment_ratio_orig"].mean(),
        ],
    },
    index=["Lines of Code", "Characters", "Comment Lines", "Comment Ratio"],
)
code_structure_latex = generate_latex_table(
    code_structure_data,
    "Code Structure Metrics Across Different Layers",
    "tab:code_structure",
)

# Print LaTeX code for all tables
print(response_rates_latex)
print(code_quality_latex)
print(code_complexity_latex)
print(code_structure_latex)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Columns: 104 entries, task_type to effort_orig
dtypes: bool(1), float64(69), int64(4), object(30)
memory usage: 201.5+ KB
None
\begin{table}[h]
\centering
\caption{Response Rates Across Different Layers}
\label{tab:response_rates}
\begin{tabular}{lcccc}
\hline
Metric & Layer 1 & Layer 10 & Layer 23 & Original \\
\hline
Response Rate & 0.3000 & 0.6680 & 0.6720 & 0.6720 \\
Sample Size & 169.0000 & 169.0000 & 169.0000 & 169.0000 \\
\hline
\end{tabular}
\end{table}

\begin{table}[h]
\centering
\caption{Code Quality Metrics Across Different Layers}
\label{tab:code_quality}
\begin{tabular}{lcccc}
\hline
Metric & Layer 1 & Layer 10 & Layer 23 & Original \\
\hline
Syntactic Correctness & 0.0059 & 0.0357 & 0.0179 & 0.1190 \\
Function Name Consistency & 0.6568 & 1.0000 & 0.9821 & 0.9940 \\
\hline
\end{tabular}
\end{table}

\begin{table}[h]
\centering
\caption{Code Complexity Metrics Across Different Layers}
\label{tab:code_c